# Introduction
In this notebook, we model PBS (Proposer Builder Seperation) and the incentives for validators, builders and provers when implemented to existing ZKEVMs.

# Basic Setup
In this baseline setup, all proposers build local blocks and one is randomly chosen according to the rules of PBS (Proposer Builder Seperation). First we define a simple `Validator` class. Validators have some notion of internal profit i.e. the value that has accrued to them over time. 


In [1]:
class Validator:
    def __init__(self):
        self.value = 32

In [2]:
constants = {
    "NUMBER_OF_VALIDATORS": 10,
    "AVG_BLOCK_REWARD": 0.025
}

cadCAD simulations are set up as a repeating pattern of state updates and policies. For this basic initial simulation, we will first update the state of the world by initialising a validator set. <say more stuff about how the sim actually works>. Each round, a validator is chosen at random from the set of all validators to propose the next block and receive the reward. Assume there is no MEV in this world, and that block rewards are fixed.

In [3]:
import pandas as pd
import cadCAD
import random

initial_state = {
    "validator_set": [Validator()] * constants["NUMBER_OF_VALIDATORS"]
}

def choose_next_block_proposer(params, substep, state_history, previous_state, policy_input):
    if not previous_state:
        previous_state = initial_state

    return "block_proposer", random.choice(previous_state["validator_set"]).index()

def update_internal_value(params, substep, state_history, previous_state, policy_input):
    block_proposer = policy_input["block_proposer"]
    previous_state["validator_set"][block_proposer].value += constants.AVG_BLOCK_REWARD

    return "rewards", previous_state["validator_set"][block_proposer]

psub = [{
    "policies": {
        "gas_used": choose_next_block_proposer # step 1
    },
    "variables": {
        "rewards": update_internal_value # step 2
    }
}]

Now we set up the parameters of the simulation. Let's run it for 10 blocks.

In [4]:
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

sim_config_dict = {
    "N": 1,
    "T": range(10),
}

c = config_sim(sim_config_dict)
exp = Experiment()

exp.append_model(
    initial_state = initial_state,
    state_update_blocks = psub,
    sim_configs = c
)



Executing the simulation...

In [5]:
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor
import pandas as pd

exec_mode = ExecutionMode()
local_mode_ctx = ExecutionContext(context=exec_mode.local_mode)

simulation = Executor(exec_context=local_mode_ctx, configs=exp.configs)
raw_system_events, tensor_field, sessions = simulation.execute()

simulation_result = pd.DataFrame(raw_system_events)


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

cadCAD Version: 0.4.28
Execution Mode: local_proc
Simulation Dimensions:
Entire Simulation: (Models, Unique Timesteps, Params, Total Runs, Sub-States) = (1, 10, 0, 1, 1)
     Simulation 0: (Timesteps, Params, Runs, Sub-States) = (10, 0, 1, 1)
Execution Method: local_simulations
Execution Mode: single_threaded
Total execution time: 0.01s


/Users/jolenedunne/PycharmProjects/pbs-sims/venv/lib/python3.9/site-packages/cadCAD/utils/__init__.py:124: FutureWarning: The use of a dictionary to describe Partial State Update Blocks will be deprecated. Use a list instead.
  warnings.warn(
/Users/jolenedunne/PycharmProjects/pbs-sims/venv/lib/python3.9/site-packages/cadCAD/utils/__init__.py:124: FutureWarning: The use of a dictionary to describe Partial State Update Blocks will be deprecated. Use a list instead.
  warnings.warn(
